In [1]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [2]:
# Dataset: [x₁, x₂, x₃], label
data = [
    ([0.1, 0.2, 0.3], 0),
    ([0.2, 0.1, 0.2], 0),
    ([0.3, 0.3, 0.1], 0),
    ([0.7, 0.8, 0.9], 1),
    ([0.8, 0.7, 0.8], 1),
    ([0.9, 0.9, 0.7], 1)
]

In [17]:
# Define the VQC with 2 layers
def create_vqc(features, params):
    qc = QuantumCircuit(3, 1)  # 3 qubits, 1 classical bit
    # Data encoding
    qc.ry(features[0] * np.pi, 0)
    qc.ry(features[1] * np.pi, 1)
    qc.ry(features[2] * np.pi, 2)
    # Layer 1
    qc.ry(params[0], 0)  # θ₁
    qc.ry(params[1], 1)  # θ₂
    qc.ry(params[2], 2)  # θ₃
    # qc.cx(0, 1)
    # qc.cx(1, 2)
    # Layer 2
    # qc.ry(params[3], 0)  # θ₄
    # qc.ry(params[4], 1)  # θ₅
    # qc.ry(params[5], 2)  # θ₆
    # qc.cx(0, 1)
    # qc.cx(1, 2)
    # Measure q0
    qc.measure(0, 0)
    return qc

In [4]:
# Simulate the circuit
def run_circuit(features, params, shots=1024):
    qc = create_vqc(features, params)
    simulator = AerSimulator()
    job = simulator.run(qc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    prob_0 = counts.get('0', 0) / shots  # P(|0⟩) for q0
    return prob_0

In [5]:
# Cost function
def cost_function(params):
    total_cost = 0
    for features, label in data:
        prob_0 = run_circuit(features, params)
        target_prob = 1.0 if label == 0 else 0.0
        total_cost += (prob_0 - target_prob) ** 2
    return total_cost / len(data)

In [18]:
# Optimize
initial_params = np.random.uniform(0, np.pi, 6)  # Random initial θ₁ to θ₆
print(f"Initial parameters: {initial_params}")
result = minimize(cost_function, initial_params, method='COBYLA', tol=1e-6)
optimal_params = result.x
print(f"Optimized parameters: {optimal_params}")
print(f"Final cost: {result.fun:.6f}")

Initial parameters: [0.95528593 1.60704984 3.09908246 3.13943764 3.00864309 1.7715846 ]
Optimized parameters: [2.34379873e-04 1.34299183e+00 2.82457457e+00 3.10224662e+00
 2.97384260e+00 2.79644203e+00]
Final cost: 0.017190


In [20]:
# Test the trained circuit
for features, label in data:
    prob_0 = run_circuit(features, optimal_params)
    prediction = 1 if prob_0 < 0.5 else 0
    print(f"Features: {features}, True Label: {label}, P(|0⟩): {prob_0:.4f}, Predicted: {prediction}")

Features: [0.1, 0.2, 0.3], True Label: 0, P(|0⟩): 0.9756, Predicted: 0
Features: [0.2, 0.1, 0.2], True Label: 0, P(|0⟩): 0.8965, Predicted: 0
Features: [0.3, 0.3, 0.1], True Label: 0, P(|0⟩): 0.7773, Predicted: 0
Features: [0.7, 0.8, 0.9], True Label: 1, P(|0⟩): 0.1963, Predicted: 1
Features: [0.8, 0.7, 0.8], True Label: 1, P(|0⟩): 0.0938, Predicted: 1
Features: [0.9, 0.9, 0.7], True Label: 1, P(|0⟩): 0.0156, Predicted: 1
